In [2]:
from benchmark import bbobbenchmarks as bn
import os
import os.path
import warnings
from cProfile import label

import numpy as np
from bokeh.embed import components
from SALib.sample import saltelli,finite_diff, fast_sampler, latin
from SALib.analyze import delta, dgsm, fast, morris, pawn, rbd_fast, sobol
from SALib.plotting.bar import plot as barplot
from SALib.plotting.hdmr import plot as hdmrplot
from SALib.plotting.morris import (
    covariance_plot,
    horizontal_bar_plot,
    sample_histograms,
)
from SALib.sample.morris import sample
from SALib.util import read_param_file
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from tqdm import tqdm

warnings.filterwarnings("ignore")
use_graph_tool = False
try:
    from graph_tool import draw
    from graph_tool.all import *

    import plotting.network_tools as nt

    use_graph_tool = True
except ModuleNotFoundError:
    # Error handling (ignore and do not use)
    pass
except ImportError:
    pass

import json
import shutil
import textwrap
from datetime import datetime

import matplotlib.pyplot as pl
import pandas as pd
from bokeh.plotting import figure, output_file, save
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

import plotting.data_processing as dp
import plotting.interactive_plots as ip
from plotting.plotting import TS_CODE, make_plot, make_second_order_heatmap
from bokeh.plotting import ColumnDataSource, figure, show

dim = 5
problem = {
    "num_vars": dim,
    "names": ["X" + str(x) for x in range(dim)],
    "bounds": [[-5.0, 5.0]] * dim,
}
fun, opt = bn.instantiate(3, iinstance=1)


In [17]:
X_sobol = saltelli.sample(problem, N=500, calc_second_order=True)
z_sobol =  np.asarray(list(map(fun, X_sobol)))

sa = sobol.analyze(problem, z_sobol, print_to_console=False, seed=0, calc_second_order=True)

df = sa.to_df()
sa_dict = dp.format_salib_output(sa, "problem", pretty_names=None)

#p = ip.plot_dict(sa_dict["problem"], min_val=0, top=5, log_axis=True)
#p.sizing_mode = "scale_width"
#p.title = "Sobol first order and total sensitivities"
#show(p)

g = nt.build_graph(sa_dict["problem"], sens="ST", top=5, edge_cutoff=0.002, edge_width=300)
inline = True
scale = 100
for i in range(g.num_vertices()):
    g.vp["sensitivity"][i] = scale * g.vp["sensitivity"][i]

#print(g.ep["second_sens"])
#for i in range(g.num_edges()):
#    g.ep["second_sens"][i] = scale * g.ep["second_sens"][i]

filename = "sobol_network.png"
state = graph_tool.inference.minimize_nested_blockmodel_dl(g)
draw.draw_hierarchy(
    state,
    vertex_text=g.vp["param"],
    vertex_text_position="centered",
    layout="radial",
    hide=2,
    # vertex_text_color='black',
    vertex_font_size=14,
    vertex_size=g.vp["sensitivity"],
    # vertex_color='#006600',
    # vertex_fill_color='#008800',
    vertex_halo=True,
    vertex_halo_color="#b3c6ff",
    vertex_halo_size=g.vp["confidence"],
    edge_pen_width=g.ep["second_sens"],
    #subsample_edges=100,
    output_size=(500, 500),
    inline=inline,
    output=filename,
)

Created a graph with 5 vertices and 2 edges.
Vertices are the top 5 ST values greater than 0.01.
Only S2 values (edges) greater than 0.002 are included.


(<VertexPropertyMap object with value type 'vector<double>', for Graph 0x19c81f8e0, at 0x19c64d9a0>,
 <Graph object, directed, with 6 vertices and 5 edges, at 0x19c7cc490>,
 <VertexPropertyMap object with value type 'vector<double>', for Graph 0x19c7cc490, at 0x19e20e0d0>)

In [3]:


p = ip.plot_second_order(sa_dict["problem"], top=5)
show(p)
#p.sizing_mode = "scale_width"

In [9]:
X_dgsm = finite_diff.sample(problem, N=500)
z_dgsm =  np.asarray(list(map(fun, X_dgsm)))
Si = dgsm.analyze(problem, X_dgsm, z_dgsm, print_to_console=False)
df = Si.to_df()
df.reset_index(inplace=True)
df



,index,vi,vi_std,dgsm,dgsm_conf
0,X0,2275.678564,2288.914236,0.951538,202.936666
1,X1,3922.314239,4407.127111,1.640052,408.104186
2,X2,7417.641421,8943.520030,3.101567,782.060618
3,X3,21708.055538,36595.649018,9.076872,3409.585865
4,X4,29531.184240,41531.071951,12.347987,3714.964003


In [12]:

df = df.sort_values(by=["vi"], ascending=False)
dftop = df.iloc[:5]
p = figure(
    x_range=dftop["index"],
    plot_height=300,
    plot_width=400,
    toolbar_location="right",
    title="DGSM vi"
)
p = ip.plot_errorbar(
    dftop,
    p,
    base_col="vi",
    error_col="vi_std",
    label_x="vi",
    label_y="vi std.",
)
#p.sizing_mode = "scale_width"
show(p)

In [3]:
Si = pawn.analyze(
    problem, X_sobol, z_sobol, S=10, print_to_console=False, seed=1
)
df = Si.to_df()
df = df.sort_values(by=["median"], ascending=False)
df.reset_index(inplace=True)
dftop = df.iloc[:5]
# p = figure(x_range=dftop['index'], plot_height=200, plot_width=20*top, toolbar_location="right", title="Pawn", tools=plottools)
p = ip.plot_pawn(dftop, p)
#p.sizing_mode = "scale_width"
show(p)

In [4]:
reg = LinearRegression().fit(X_sobol, z_sobol)
print( reg.coef_ )
print(reg.intercept_)

[  5.088961    -7.49113946 -12.46331687  -1.02765228 -39.37541462]
-142.43415961200586
